# Fantasy Football - Optimal Drafting Guide and Predictions

### Goals:  
1. Create a PPR drafting strategy guide.
    - Round by round analysis
    - General drafting principles
      
2. Create 2023 player ppr projections and tier list

In [1]:
# Imports
import pandas as pd
import numpy as np
import time

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

In [3]:
df = pd.read_csv('season.csv',index_col=0)
df = df.sort_values('year').reset_index(drop=True)

In [4]:
df.head()

,player,rk,team,pos,age,g,gs,cmp,pass_att,pass_yds,...,standard_pts,ppr_pts,vbd,pos_rank,year,avg_draft_pos,avg_draft_pos_ppr,adp_by_pos,success,round
0,Aaron Rodgers,3,GNB,QB,33,16,16,401.0,610.0,4428.0,...,380.0,380.0,119.0,1,2016,32.3,38.7,2,1.0,4.0
1,Harry Douglas,313,TEN,WR,32,11,2,0.0,0.0,0.0,...,21.0,36.0,0.0,116,2016,305.0,284.0,119,0.0,15.0
2,Jeff Janis,296,GNB,WR,25,16,1,0.0,0.0,0.0,...,25.0,36.1,0.0,115,2016,248.0,238.0,99,0.0,15.0
3,Cody Core,322,CIN,WR,22,8,4,0.0,0.0,0.0,...,20.0,37.0,0.0,114,2016,350.0,350.0,164,0.0,15.0
4,Bennie Fowler,286,DEN,WR,25,13,0,0.0,0.0,0.0,...,27.0,37.5,0.0,113,2016,271.0,248.0,105,0.0,15.0
